# Librerías

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.language_models.llms import LLM
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import Field
from sqlalchemy import create_engine, text
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from typing import Optional, List, Any
from langchain.schema import Document
from transformers import pipeline
from flask import Flask, request
from datetime import datetime
from peft import PeftModel
import gradio as gr
import pandas as pd
import gradio as gr
import numpy as np
import requests
import uuid
import json
import torch
import gc
import re
import os
from dotenv import load_dotenv

d:\Python_Envs\torch311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Python_Envs\torch311\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


# Microsoft Phi-3 como modelo

In [2]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [3]:
# Modelo local 
model_path = "D:/LLM Models/microsoft-phi-3-mini-4k-instruct"

# LoRA adapters generados con el archivo LL_Fine_Tune_agave.ipynb
lora_path = "D:/LLM Models/agave_V001/agave_baseline_phi3_V01"   

# 8 bits de cuantizacion
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
)

# Usamos el tokenizer de Phi-3
tokenizer = AutoTokenizer.from_pretrained(model_path)
base_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto",
)



`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.12s/it]


In [4]:
# Implementamos los pesos LoRA a Phi-3
model = PeftModel.from_pretrained(base_model, lora_path)

# Colocamos el  modelo en evaluacion para que no cambie los pesos por cada prompt que
# se le envíe
model.eval()

# Definimos el pipeline de nuevo
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    return_full_text=False,
)

Device set to use cuda:0


In [5]:
def gen_prompt(tokenizer, sentence):
    converted_sample = [{"role": "user", "content": sentence}]
    prompt = tokenizer.apply_chat_template(
        converted_sample, tokenize=False, add_generation_prompt=True
    )
    return prompt

In [6]:


def generate(model, tokenizer, prompt, max_new_tokens=256, skip_special_tokens=False):
  
  tokenized_input = tokenizer(
  prompt, add_special_tokens=False, return_tensors="pt").to(model.device)
  model.eval()
  gen_output = model.generate(**tokenized_input,
  eos_token_id=tokenizer.eos_token_id,
  max_new_tokens=max_new_tokens)
  output = tokenizer.batch_decode(gen_output, skip_special_tokens=skip_special_tokens)

  return output[0]

# Sistemas RAG

#### Vector Store

In [7]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    encode_kwargs={"normalize_embeddings": True}
)

vs = FAISS.load_local(
    "C:/Users/Delbert/Documents/Maestria/Proyecto Integrador/Avance 1/Tecnologico-Monterrey-Proyecto-Integrador-equipo-36/Baseline/rag_faiss_store",
    embeddings,
    allow_dangerous_deserialization=True
)

print("Total docs:", len(vs.docstore._dict))

C:\Users\Delbert\AppData\Local\Temp\ipykernel_7520\2113877534.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Total docs: 620196


### Modelo Phi-3 como un custom LLM

In [8]:
class Phi3ChatTemplateLLM(LLM):
    model: Any = Field(exclude=True)
    tokenizer: Any = Field(exclude=True)
    max_new_tokens: int = 256
    skip_special_tokens: bool = True

    @property
    def _llm_type(self) -> str:
        return "phi3-chat-template"

    def _call(self, prompt: str, stop: Optional[List[str]] = None, **kwargs: Any) -> str:
        
        chat_prompt = gen_prompt(self.tokenizer, prompt)
        self.model.eval()
        
        with torch.inference_mode():
            out = generate(
                self.model,
                self.tokenizer,
                chat_prompt,
                max_new_tokens=self.max_new_tokens,
                skip_special_tokens=self.skip_special_tokens
            )
        
        return out


custom_llm = Phi3ChatTemplateLLM(model=model, tokenizer=tokenizer, max_new_tokens=256, skip_special_tokens=True)

In [9]:
my_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
       """Eres un asistente experto en el picudo del agave (Scyphophorus acupunctatus) para CNIT. Tu objetivo es ayudar a productores y técnicos con información práctica sobre esta plaga.

INSTRUCCIONES CRÍTICAS:
1. Responde ÚNICAMENTE basándote en el CONTEXTO proporcionado
2. Si el CONTEXTO no contiene la información: responde exactamente "No cuento con esa información. ¿Puedo ayudarte en algo más?"
3. NUNCA inventes, supongas o uses conocimiento externo
4. Mantén respuestas concisas con máximo de 4 oraciones
5. Usa lenguaje natural y accesible para agricultores
6. NO menciones campos técnicos como: trap_id, severity_encoded, fechas específicas de base de datos
7. Cuando hables de ubicaciones, usa nombres de municipios/estados, no coordenadas

CONTEXTO DISPONIBLE:
{context}

PREGUNTA DEL USUARIO:
{question}

RESPUESTA DE ASISTENTE:"""
    ),
)


retriever = vs.as_retriever(search_kwargs={"k": 5})

rag_chain = RetrievalQA.from_chain_type(
    llm=custom_llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": my_prompt},
    return_source_documents=False
)


# Conexión a base de datos de reportes en GCP

In [10]:
# Cargamos variables de entorno
load_dotenv(dotenv_path="chatbot.env")

# Guardamos variables de entorno para conectarnos a la BDD
USER = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD_DB')
HOST = os.getenv('HOST_DB')
DBNAME = 'postgres'#os.getenv('DBNAME')
PORT = 5432

# Instanciamos el engine para conectarnos a nuestra BDD en GCP
engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}")

# Confirmamos que nos podamos conectar
try:
    with engine.connect() as conn:
        print("✅ Conexión exitosa con super_user")

except Exception as e:
    print("❌ Error de conexión:", e)

✅ Conexión exitosa con super_user


In [11]:
def get_total_cases_count():
    try:
        with engine.connect() as conn:
            total_cases = conn.execute(text("SELECT COUNT(*) FROM reportes_chatbot;")).scalar()
        return total_cases

    except:
        print("Error conectándose a la BDD.")

In [12]:
def insert_report_into_bdd(data):

    try:
        
        query = text("""
        INSERT INTO reportes_chatbot (
            report_id,
            case_number,
            lat,
            lon,
            address,
            photo_url,
            user_risk_assesment,
            user_lot_description,
            contact_phone,
            price,
            current_state
        ) VALUES (
            :report_id,
            :case_number,
            :lat,
            :lon,
            :address,
            :photo_url,
            :user_risk_assesment,
            :user_lot_description,
            :phone_number,
            :price,
            'Nuevo'
        );""")
        
        try:
            with engine.begin() as conn:
                conn.execute(query, data)
            print(f"✅ Reporte insertado: {data.get('case_number')} ({data.get('report_id')})")
            return True, data.get('case_number')

        except Exception as e:
            print(f"❌ Error al insertar el reporte {data.get('case_number')}: {e}")
            return False
        
    except:
        print("❌ No se pudo insertar el reporte a la BDD, se procede a desechar.")
        return False

In [13]:
def get_case_state(case_suffix: str):

    # Aseguramos el formato correcto con el prefijo 'CASO-'
    case_number = f"CASO-{case_suffix.strip()}"

    query = text("""
        SELECT current_state
        FROM reportes_chatbot
        WHERE case_number = :case_number;
    """)

    try:
        with engine.connect() as conn:
            result = conn.execute(query, {"case_number": case_number}).fetchone()

            if result is None:
                return False, None
            else:
                return True, result[0]

    except Exception as e:
        return f"⚠️ Error al consultar la base de datos: {e}"

# Conexión a Telegram

In [14]:
TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
BASE_URL = f"https://api.telegram.org/bot{TOKEN}"

In [15]:
# Diccionario que guardara la informacion temporalmente
# antes de escribirla a la BDD
user_session = {}
case_counter = get_total_cases_count()

In [16]:
def send_menu(chat_id):

    menu_message = (
        "👋 *Bienvenido, será un gusto atenderte.*\n\n"
        "¿Qué te interesa llevar a cabo?\n\n"
        "A. *Hacer un reporte de un predio infectado*\n\n"
        "B. *Preguntar por el seguimiento a un reporte anterior.*\n\n"
        "C. *Preguntar generalidades sobre el muestreo de gorgojos.*\n\n"
        "_Actualmente, solo estas 3 opciones están disponibles._"
    )

    payload = {
        "chat_id": chat_id,
        "text": menu_message,
        "parse_mode": "Markdown"
    }

    try:
        response = requests.post(f"{BASE_URL}/sendMessage", json=payload)
        
        if response.status_code == 200:
            print(f"✅ Menú enviado correctamente a {chat_id}")
            
        else:
            print(f"⚠️ Error enviando menú: {response.text}")
            
            
    except Exception as e:
        print(f"❌ Error al enviar menú: {e}")

In [17]:
def report_message_flow(sender_id, state, message_data):

    # Inicializamos variables
    response_text = ""
    next_state = state
    
    current_case_state_dict = {
        "Nuevo": " el caso fue recibido y será investigado por un experto en control de plagas de picudo.",
        "En proceso" : " el caso fue asignado a un experto en control de plagas de picudo.",
        "En campo"   : " el caso se encuentra siendo investigado en el sitio reportado",
        "Falsa alerta" : " el caso se cataloga como una falsa alerta con base en la investigación realizada.",
        "Verdadera alerta" : " se confirma una infestación de picudo y se tomaron acciones adecuadas para erradicarlo.",
        "Concluido" : " se llevó a cabo la investigación, erradicación y control del picudo."
    }
    
    # Detectamos si el usuario ha compartido su numero de telefono
    contact = message_data.get("contact", {})
    phone_number = contact.get("phone_number")

    # Detectamos el contenido del mensaje
    text = message_data.get("text", "").strip().lower() if "text" in message_data else ""
    location = message_data.get("location", {})
    photo = message_data.get("photo", [])
    lat = location.get("latitude")
    lon = location.get("longitude")
    
    # Guardamos el contacto si este es publico
    if phone_number:
        user_session[sender_id]["data"]["phone_number"] = phone_number


    # Extraemos la URL de la foto (si existe)
    photo_url = None
    if photo:
        file_id = photo[-1]["file_id"]  # La última suele ser la de mejor resolución
        photo_url = f"https://api.telegram.org/file/bot{TOKEN}/{file_id}"


    # ============================================================
    # Saludo inicial para reportar un predio con problemas
    # ============================================================
    
    if state == "saludo":
        if text in ["a", "🅰️"]:
            response_text = (
                "Gracias por tu proactividad 🌾.\n Primero, envía la **ubicación** del lote afectado usando el ícono 📍."
            )
            next_state = "ask-location"

        elif text in ["b", "🅱️"]:
            response_text = (
                "Claro, por favor indícame tu número de caso.\n\nEl formato del caso debe contener el año y número de reporte.\n"
                "\nEste es un ejemplo de un número de caso válido: 2025-0001"
            )
            next_state = "ask-case-file"
            
        elif text in ["c"]:
            response_text = (
                "Perfecto. Puedes escribir hasta 3 preguntas y te responderé con ayuda del asistente inteligente."
            )
            next_state = "NLP-Chatbot-1"

    # ============================================================
    # Preguntamos por el número de caso en caso de que se haya seleccionado esa opcion.
    # ============================================================
    elif state == "ask-case-file":
        regex_pattern = r"^(20\d{2})-(\d{4})$"
        
        if text:
            text = text.strip()
            valid_case_format = re.match(regex_pattern, text)
            
            if valid_case_format:
                
                case_success, current_case_state = get_case_state(text)
                
                if not case_success:
                    response_text = (f"No existe un caso con el correlativo {text} en nuetra base de datos.\n\n"
                                      "Por lo anterior, te instamos a realizar un reporte si conoces alguna situación o predio que \n"
                                      "presente un riesgo de infestación.")
                    
                    next_state = 'restart'
                    
                else:
                    
                    description = current_case_state_dict.get(
                            current_case_state, 
                            "no se tiene una descripción disponible para este estado."
                        )
                    response_text = (f"El caso {text} se encuentra en el estado: {current_case_state} "
                                     f"\n\nEsto significa que {description}\n\nPara reiniciar el chat, escribe algún texto de nuevo, por favor.")
                    
                    next_state = 'restart'
                
            else:
                response_text = ("No reconozco ese formato de caso.\nPor favor, envía tu caso usando el formato `año`-`# de caso`.\n\nEjemplo: 2025-0001")
        
        else:
            response_text = "⚠️ No detecté un número de caso. Por favor, envía el número siguiendo el formato 2025-<Numero de caso en 4 digitos>."
        
    # ============================================================
    # Preguntamos por la ubicación. Si esta se envía, generamos un número de caso unico y un
    # numero de caso relativo para evitar duplicar reportes
    # ============================================================
    
    elif state == "ask-location":
        if lat and lon:
            
            # Usaremos el contador global de casos actuales en la BDD
            case_counter = get_total_cases_count() + 1

            # Generamos un id de reporte unico al tener ubicacion
            report_id = f"report_{uuid.uuid4()}"
            user_session[sender_id]["data"]["report_id"] = report_id

            # Generamos el número de caso legible
            year = datetime.now().year
            case_number = f"CASO-{year}-{case_counter:04d}"
            user_session[sender_id]["data"]["case_number"] = case_number
            
            # Guardamos la ubicacion de latitud y longitud
            user_session[sender_id]["data"]["lat"] = lat
            user_session[sender_id]["data"]["lon"] = lon
            
            response_text = (
                "📍 *Ubicación recibida correctamente.*\n\n Ahora, por favor envía una **foto** del lote o planta afectada. 📸"
            )
            next_state = "ask_photo"
        else:
            response_text = (
                "⚠️ No detecté una ubicación válida. Por favor, usa el botón de 📍 para enviarla correctamente."
            )
            
    # ============================================================
    # Pedimos una fotografía sobre lo reportado por el usuario
    # ============================================================
    elif state == "ask_photo":
        if photo_url:
            user_session[sender_id]["data"]["photo_url"] = photo_url
            response_text = (
                "📸 *Foto recibida correctamente.*\n" 
                "Por favor, ahora describe las condiciones del predio. Es de ayuda conocer si el predio se encuentra\n\n"
                "1. Abandonado\n2. En mal estado.\n3. Infestado por picudos.\n"
                "\nCualquier informacion adicional en tu descripción es de ayuda "
            )
            next_state = "ask_lot_description"
        else:
            response_text = "⚠️ No se detectó una foto. Por favor envía una imagen del lote afectado."


    # ============================================================
    # Luego preguntamos por una descripción detallada del usuario
    # ============================================================
    elif state == "ask_lot_description":    
        if text:
            user_session[sender_id]["data"]["user_lot_description"] = text
            
            response_text = (
                "Gracias por tu descripción del predio. Ahora, clasifica el riesgo que representa para tí este predio de alguna de las siguientes formas:\n"
                "1. Alto\n2. Medio\n3. Bajo"
            )
            
            next_state = 'ask_risk'
        
        else:
            response_text = "⚠️ No se detectó una respuesta válida. Por favor, describe el estado del predio usando oraciones completas."
    
    # ============================================================
    # Ahora preguntamos el riesgo que el usuario percibe
    # ============================================================
    elif state == "ask_risk":
        
        if text in ["alta", "media", "baja", "alto", "medio", "bajo", "1", "2", "3"]:
            
            if text in ["1"]:
                text = "Alto"
            if text in ["2"]:
                text = "Medio"
            if text in ["3"]:
                text = "Bajo"
            
            user_session[sender_id]["data"]["user_risk_assesment"] = text.capitalize()
            data = user_session[sender_id]["data"]
            
            response_text = (
                f"✅ Gracias por tu reporte.\n"
                f"📍 Ubicación: ({data['lat']}, {data['lon']})\n"
                f"📸 Foto: Confirmada\n"
                f"🚨 Riesgo: {data['user_risk_assesment']}\n"
                f"💬 Descripción: Recibida\n\n"
                "¿Esta información es correcta? Responde con *Sí* o *No*."
            )
            next_state = "confirmation"
        else:
            response_text = "Por favor indica el nivel de riesgo como: Alta, Media o Baja."

    # ============================================================
    # Ahora informamos sobre el reporte dando el correlativo de cada caso
    # ============================================================
    elif state == "confirmation":
        
        # Si el reporte es correcto, lo confirmamos
        if re.search(r'^\s*s[ií]\s*$', text):
        
            response_text = "🌾 Tu reporte ya fue registrado. ¡Gracias por tu colaboración!"
            next_state = "restart"
            
            inserted, case = insert_report_into_bdd(user_session[sender_id]["data"])
            
            if inserted:
                response_text = response_text + f"\n\nAdicionalmente, se creó el caso {case} por si quisieras consultar el estado del mismo."
            
        # Si nos dice que no es correcto, desechamos el reporte
        elif re.search(r'^\s*no\s*$', text):
            response_text = "❌ Entendido. Tu reporte no se registrará.\nPara repetir el reporte, o hacer otro, puedes volver a escribir sobre este mismo chat."
            next_state = "restart"
        
        # Si nos responde algo que no sabemos que es
        else:
            response_text = "Por favor responde únicamente con *Sí* o *No*."
            

    # Else que solo existe por si no capturamos la logica de la respuesta
    else:
        response_text = "No entendí tu respuesta. Por favor intenta nuevamente."

    return response_text, next_state

In [18]:
def send_message(chat_id, text):
    """
    Envía un mensaje de texto al usuario usando la API de Telegram.
    """
    payload = {
        "chat_id": chat_id,
        "text": text,
        "parse_mode": "Markdown"
    }
    try:
        response = requests.post(f"{BASE_URL}/sendMessage", json=payload)
        if response.status_code != 200:
            print(f"⚠️ Error enviando mensaje: {response.text}")
    except Exception as e:
        print(f"❌ Error al enviar mensaje: {e}")

In [19]:
# Esta funcion la usamos porque telegram necesita que hagamos scape 
# de algunos caracteres como puntos, signos de exclamacion, etc.
def escape_markdown_v2(text):
    return re.sub(r'([_*\[\]()~`>#+\-=|{}.!])', r'\\\1', text)

In [20]:
app = Flask(__name__)

@app.route("/webhook", methods=["POST"])
def telegram_webhook():
    """
    Webhook principal para manejar mensajes entrantes desde Telegram.
    Procesa texto, fotos y ubicaciones, mantiene el flujo de conversación
    y responde usando la API de Telegram.
    """
    data = request.get_json()

    if "message" not in data:
        return "ok", 200

    message = data["message"]
    chat_id = message["chat"]["id"]
    username = message["chat"].get("username", "Desconocido")

    print("="*50)
    print(f"Mensaje recibido de @{username} | chat_id={chat_id}")
    print("="*50)

    # Si es la primera vez que el usuario escribe
    if (chat_id not in user_session) or (user_session[chat_id]["state"] == "restart"):
        print("Enviando menu al usuario")
        user_session[chat_id] = {
            "state": "saludo",
            "data": {
                "lat": None,
                "lon": None,
                "address": None,
                "photo_url": None,
                "user_risk_assesment": None,
                "user_lot_description": None,
                "report_id" : None,
                "case_number" : None,
                "phone_number": None,
                "price": None
            },
        }

        # Enviamos el menú inicial
        send_menu(chat_id)
        return "ok", 200

    # Si el usuario se encuentra en modo Chatbot (NLP)
    if (user_session[chat_id]["state"] in ["NLP-Chatbot-1", "NLP-Chatbot-2", "NLP-Chatbot-3"]):
        
        print("🤖 Enviando consulta al RAG Chatbot...")
        query = message.get("text", "")
        response = rag_chain.invoke({"query": query})

        raw_output = response["result"]
        if "RESPUESTA DE ASISTENTE:" in raw_output:
            clean_output = raw_output.split("RESPUESTA DE ASISTENTE:")[-1].strip()
        else:
            clean_output = raw_output.strip()

        send_message(chat_id, escape_markdown_v2(clean_output))
        
        if user_session[chat_id]["state"] == "NLP-Chatbot-3":
            send_message(chat_id, "Has llegado al límite de preguntas consecutivas. El Chat se reiniciará ahora. Espero haber sido de ayuda.")
            user_session[chat_id]["state"] = "restart"
            
        if user_session[chat_id]["state"] == "NLP-Chatbot-2":
            user_session[chat_id]["state"] = "NLP-Chatbot-3"
            
        if user_session[chat_id]["state"] == "NLP-Chatbot-1":
            user_session[chat_id]["state"] = "NLP-Chatbot-2"
            
        
        return "ok", 200

    # En caso contrario, seguimos el flujo de reporte
    current_state = user_session[chat_id]["state"]
    response_text, next_state = report_message_flow(chat_id, current_state, message)
    user_session[chat_id]["state"] = next_state

    send_message(chat_id, response_text)
    return "ok", 200

In [ ]:
if __name__ == "__main__":
	app.run(port=3000, debug=False, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3000
Press CTRL+C to quit


Mensaje recibido de @Desconocido | chat_id=6940133416
Enviando menu al usuario


127.0.0.1 - - [08/Nov/2025 21:47:10] "POST /webhook HTTP/1.1" 200 -


✅ Menú enviado correctamente a 6940133416
Mensaje recibido de @Desconocido | chat_id=6940133416


...

127.0.0.1 - - [08/Nov/2025 21:56:19] "POST /webhook HTTP/1.1" 200 -


✅ Menú enviado correctamente a 7206382205
Mensaje recibido de @Desconocido | chat_id=7206382205


127.0.0.1 - - [08/Nov/2025 21:56:22] "POST /webhook HTTP/1.1" 200 -


Mensaje recibido de @Desconocido | chat_id=7206382205


127.0.0.1 - - [08/Nov/2025 21:56:30] "POST /webhook HTTP/1.1" 200 -
